In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
import numpy as np
import cv2
import os

In [4]:
train_dir = "/content/drive/MyDrive/capstone/data/train_data"
validation_dir = "/content/drive/MyDrive/capstone/data/validation_data"

In [5]:
train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

In [6]:
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size = 20,
    class_mode = 'binary',
    target_size = (224, 224))

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    batch_size = 20,
    class_mode = 'binary',
    target_size = (224, 224))

Found 59 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


In [8]:
base_model = VGG16(
    input_shape = (224, 224, 3),
    include_top = False,
    weights = 'imagenet')

58900480/58889256 [==============================] - 1s 0us/step


In [9]:
for layer in base_model.layers:
    layer.trainable = False

In [10]:
x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
model.compile(
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001),
    loss = 'binary_crossentropy',
    metrics = ['acc'])

In [13]:
vgghist = model.fit(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch = 3,
    epochs = 10)

Epoch 1/10
3/3 [==============================] - 39s 13s/step - loss: 1.9610 - acc: 0.5085 - val_loss: 1.0867 - val_acc: 0.5000
Epoch 2/10
3/3 [==============================] - 2s 566ms/step - loss: 1.1657 - acc: 0.5254 - val_loss: 0.2222 - val_acc: 1.0000
Epoch 3/10
3/3 [==============================] - 2s 568ms/step - loss: 0.3192 - acc: 0.8475 - val_loss: 0.1986 - val_acc: 0.9444
Epoch 4/10
3/3 [==============================] - 2s 541ms/step - loss: 0.2447 - acc: 0.9153 - val_loss: 0.1035 - val_acc: 1.0000
Epoch 5/10
3/3 [==============================] - 2s 560ms/step - loss: 0.4403 - acc: 0.7119 - val_loss: 0.5844 - val_acc: 0.6667
Epoch 6/10
3/3 [==============================] - 2s 562ms/step - loss: 0.2662 - acc: 0.8814 - val_loss: 0.0644 - val_acc: 1.0000
Epoch 7/10
3/3 [==============================] - 2s 538ms/step - loss: 0.1292 - acc: 0.9661 - val_loss: 0.0661 - val_acc: 1.0000
Epoch 8/10
3/3 [==============================] - 2s 524ms/step - loss: 0.0710 - acc: 0.983

In [ ]:
model.save('vgg.h5')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
cap = cv2.VideoCapture("/content/drive/MyDrive/Abd.mp4")
no_of_frame = int(cap.get(7))

In [ ]:
from google.colab.patches import cv2_imshow

temp = []
    
i = 0
while i < no_of_frame:
    ret,frame = cap.read()
    frame = cv2.resize(frame,(224,224))
    img = np.asarray(frame)
    img = np.expand_dims(img,axis=0)
    saved_model = load_model("/content/vgg.h5")
    output = saved_model.predict(img)
    val = sum(output[0])
    if val < 0.95:
        temp.append('Non-Pitch')
    else:
        temp.append('Pitch')
    print(str(round((i)/(no_of_frame)*100,2))+'% completed!')
    i+=1

print(temp)

0.0% completed!
0.09% completed!
0.17% completed!
0.26% completed!
0.34% completed!
0.43% completed!
0.51% completed!
0.6% completed!
0.68% completed!
0.77% completed!
0.85% completed!
0.94% completed!
1.02% completed!
1.11% completed!
1.2% completed!
1.28% completed!
1.37% completed!
1.45% completed!
1.54% completed!
1.62% completed!
1.71% completed!
1.79% completed!
1.88% completed!
1.96% completed!
2.05% completed!
2.13% completed!
2.22% completed!
2.31% completed!
2.39% completed!
2.48% completed!
2.56% completed!
2.65% completed!
2.73% completed!
2.82% completed!
2.9% completed!
2.99% completed!
3.07% completed!
3.16% completed!
3.25% completed!
3.33% completed!
3.42% completed!
3.5% completed!
3.59% completed!
3.67% completed!
3.76% completed!
3.84% completed!
3.93% completed!
4.01% completed!
4.1% completed!
4.18% completed!
4.27% completed!
4.36% completed!
4.44% completed!
4.53% completed!
4.61% completed!
4.7% completed!
4.78% completed!
4.87% completed!
4.95% completed!
5.04

In [1]:
import numpy as np
import glob
cap1 = cv2.VideoCapture("/content/drive/MyDrive/Abd.mp4")
no_of_frame1 = int(cap.get(7))    

img_array = []
i = 0
while i < no_of_frame1:
    ret,frame = cap1.read()
    frame = cv2.resize(frame,(720,480))
    value = temp[i]
    frame = cv2.putText(frame, value, (25, 25), cv2.FONT_ITALIC, 1, (0, 125, 255), 3,cv2.LINE_AA)
    cv2_imshow(frame)
    img_array.append(frame)
    key = cv2.waitKey(50)
    if key == ord('q'):
        break
    i+=1
    


# for filename in glob.glob('C:/New folder/Images/*.jpg'):
#     img = cv2.imread(filename)
#     height, width, layers = img.shape
#     size = (width,height)
#     img_array.append(img)


out = cv2.VideoWriter('/content/drive/MyDrive/pitch-non-pitch-Abd.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

NameError: ignored